# Automating Code Reviews


## Schema


In [ ]:
CREATE TABLE code_reviews (
    id SERIAL PRIMARY KEY,
    review_sentiment TEXT,
    review_text TEXT NOT NULL
);

CREATE TABLE solution_files (
    id SERIAL PRIMARY KEY,
    review_id INT NOT NULL,
    file_name TEXT NOT NULL,
    file_content  TEXT NOT NULL,
    FOREIGN KEY (review_id) REFERENCES code_reviews (id) ON DELETE CASCADE
);

CREATE TABLE task_instructions (
    id SERIAL PRIMARY KEY,
    solution_id INT NOT NULL,
    task_name TEXT NOT NULL,
    task_content TEXT NOT NULL,
    task_instructions TEXT NOT NULL,
    FOREIGN KEY (solution_id) REFERENCES solution_files (id) ON DELETE CASCADE
);


## Vectorisers

### Review Sentiment and Text


In [ ]:
-- Separate

SELECT ai.create_vectorizer(
    'code_reviews'::regclass,
    destination => 'code_review_sentiment_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('review_sentiment')
);

SELECT ai.create_vectorizer(
    'code_reviews'::regclass,
    destination => 'code_review_text_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('review_text')
);


-- Combined

ALTER TABLE code_reviews
ADD COLUMN review_combined TEXT;

UPDATE code_reviews
SET review_combined = CONCAT('Sentiment: ', review_sentiment, '. Text: ', review_text);

SELECT ai.create_vectorizer(
    'code_reviews'::regclass,
    destination => 'code_review_combined_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('review_combined')
);

-- Vectoriser code reviews
SELECT ai.execute_vectorizer(57);


### Solution Files


In [ ]:
-- Separate

SELECT ai.create_vectorizer(
    'solution_files'::regclass,
    destination => 'code_solution_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('file_context')
);

SELECT ai.create_vectorizer(
    'solution_files'::regclass,
    destination => 'solution_filename_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('file_name')
);

-- Combined

ALTER TABLE solution_files
ADD COLUMN solution_combined TEXT;

UPDATE solution_files
SET solution_combined = CONCAT('File Name: ', file_name, '. File context: ', file_content);

SELECT ai.create_vectorizer(
    'solution_files'::regclass,
    destination => 'code_solution_combined_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('solution_combined')
);

-- Vectoriser solution files
SELECT ai.execute_vectorizer(63);


### Task Instructions


In [ ]:
SELECT ai.create_vectorizer(
    'task_instructions'::regclass,
    destination => 'task_title_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('title')
);

SELECT ai.create_vectorizer(
    'solution_files'::regclass,
    destination => 'code_solution_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('content')
);

SELECT ai.create_vectorizer(
    'solution_files'::regclass,
    destination => 'code_solution_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('file_name')
);

SELECT ai.create_vectorizer(
    'code_reviews'::regclass,
    destination => 'positives_code_review_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('review_positives')
);

SELECT ai.create_vectorizer(
    'code_reviews'::regclass,
    destination => 'improvements_code_review_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('review_improvements')
);

SELECT ai.create_vectorizer(
    'code_reviews'::regclass,
    destination => 'overall_code_review_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('review_overall')
);


## PSQL Functions

### Generate Code Review


### V1


In [ ]:
CREATE OR REPLACE FUNCTION generate_code_review(query_code TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    -- Retrieve relevant task instructions
    WITH relevant_tasks AS (
        SELECT title, chunk
        FROM task_instruction_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve relevant code solutions
    relevant_solutions AS (
        SELECT title, chunk
        FROM code_solution_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve relevant positives from code reviews
    relevant_positives AS (
        SELECT title, chunk
        FROM positives_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve relevant improvements from code reviews
    relevant_improvements AS (
        SELECT title, chunk
        FROM improvements_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve overall reviews from code reviews
    relevant_overall AS (
        SELECT title, chunk
        FROM overall_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    )
    -- Combine the results into a single context
    SELECT string_agg(title || ': ' || chunk, E'\n') 
    INTO context_chunks
    FROM (
        SELECT * FROM relevant_tasks
        UNION ALL
        SELECT * FROM relevant_solutions
        UNION ALL
        SELECT * FROM relevant_positives
        UNION ALL
        SELECT * FROM relevant_improvements
        UNION ALL
        SELECT * FROM relevant_overall
    ) AS combined_context;

    -- Validate the context_chunks to ensure it's not empty
    IF context_chunks IS NULL OR context_chunks = '' THEN
        RAISE EXCEPTION 'No relevant context found for the provided query_code.';
    END IF;

    -- Generate the review using the combined context
    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
            jsonb_build_object(
                'role', 'system',
                'content', 'You are Branden, a friendly and practical code reviewer. Provide clear, actionable feedback with a mix of professionalism and casual tone.'
            ),
            jsonb_build_object(
                'role', 'user',
                'content',
                'Code to Review:\n' || query_code ||
                E'\n\nReview Context:\n' || context_chunks ||
                E'\n\nRubric Instructions:\n' ||
                '1. Efficiency: Provide ratings and practical examples.\n' ||
                '2. Completeness: Mention anything missing clearly.\n' ||
                '3. Style: Give feedback on readability and structure.\n' ||
                '4. Documentation: Highlight missing or misplaced comments.\n\n' ||
                'Be conversational and include personal suggestions where possible.\n' ||
                'Ensure that relevant task objectives are successfully implemented as instructed\n' ||
                'Include code snippets where relevant'
            )
        ),
        host => 'http://ollama:11434'
    ) -> 'message' ->> 'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;

### v2

In [ ]:
CREATE OR REPLACE FUNCTION generate_code_review(query_code TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    -- Retrieve relevant task instructions
    WITH relevant_tasks AS (
        SELECT title, chunk
        FROM task_instruction_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve relevant code solutions
    relevant_solutions AS (
        SELECT title, chunk
        FROM code_solution_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve relevant positives from code reviews
    relevant_positives AS (
        SELECT title, chunk
        FROM positives_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve relevant improvements from code reviews
    relevant_improvements AS (
        SELECT title, chunk
        FROM improvements_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    -- Retrieve overall reviews from code reviews
    relevant_overall AS (
        SELECT title, chunk
        FROM overall_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    )
    -- Combine the results into a single context
    SELECT string_agg(title || ': ' || chunk, E'\n') 
    INTO context_chunks
    FROM (
        SELECT * FROM relevant_tasks
        UNION ALL
        SELECT * FROM relevant_solutions
        UNION ALL
        SELECT * FROM relevant_positives
        UNION ALL
        SELECT * FROM relevant_improvements
        UNION ALL
        SELECT * FROM relevant_overall
    ) AS combined_context;

    -- Validate the context_chunks to ensure it's not empty
    IF context_chunks IS NULL OR context_chunks = '' THEN
        RAISE EXCEPTION 'No relevant context found for the provided query_code.';
    END IF;

    -- Generate the review using the combined context
    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
            jsonb_build_object(
            'role', 'system',
            'content', 'You are Branden, a friendly and practical code reviewer. Provide clear, actionable feedback with a mix of professionalism and casual tone.'
            ),
            jsonb_build_object(
                'role', 'user',
                'content',
                'Code to Review:\n' || query_code ||
                E'\n\nReview Context:\n' || context_chunks ||
                E'\n\nRubric Instructions:\n' ||
                '1. Efficiency: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                '2. Completeness: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                '3. Style: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                '4. Documentation: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                'For each category, provide:\n' ||
                '1. Positive aspects of the submission\n' ||
                '2. Aspects that could be improved\n' ||
                '3. Overall comments\n' ||
                'Generate a clear and comprehensive review following this structure:\n' ||
                'Positive:\n' ||
                'Provide positive feedback on the submission.\n\n' ||
                'Improve:\n' ||
                'Provide constructive feedback and suggestions for improvement.\n' ||
                'Refer to the code for improvements\n\n' ||
                'Overall:\n' ||
                'Provide an overall summary of the submission.\n\n' ||
                'Ensure that relevant task objectives are successfully implemented as instructed\n' ||
                'Include code snippets where relevant\n\n' ||
                'Avoid using the following phrases and words in your review:\n' ||
                'Phrases:\n' ||
                '- A testament to…\n' ||
                '- In conclusion…\n' ||
                '- In summary…\n' ||
                '- It’s important to note/consider…\n' ||
                '- It’s worth noting that…\n' ||
                '- On the contrary…\n' ||
                'Words:\n' ||
                '- Aligns\n' ||
                '- Augment\n' ||
                '- Delve\n' ||
                '- Embark\n' ||
                '- Facilitate\n' ||
                '- Maximize\n' ||
                '- Underscores\n' ||
                '- Utilize\n' ||
                '- Efficiency\n' ||
                '- Innovation\n' ||
                '- Institution\n' ||
                '- Integration\n' ||
                '- Implementation\n' ||
                '- Landscape\n' ||
                '- Optimization\n' ||
                '- Realm\n' ||
                '- Tapestry\n' ||
                '- Transformation\n' ||
                '- Adept\n' ||
                '- Commendable\n' ||
                '- Dynamic\n' ||
                '- Efficient\n' ||
                '- Ever-evolving\n' ||
                '- Exciting\n' ||
                '- Exemplary\n' ||
                '- Innovative\n' ||
                '- Invaluable\n' ||
                '- Robust\n' ||
                '- Seamless\n' ||
                '- Synergistic\n' ||
                '- Thought-provoking\n' ||
                '- Transformative\n' ||
                '- Utmost\n' ||
                '- Vibrant\n' ||
                '- Vital\n' ||
                'Additionally, avoid using the following adverbs:\n' ||
                '- Accordingly\n' ||
                '- Additionally\n' ||
                '- Arguably\n' ||
                '- Certainly\n' ||
                '- Consequently\n' ||
                '- Hence\n' ||
                '- However\n' ||
                '- Indeed\n' ||
                '- Moreover\n' ||
                '- Nevertheless\n' ||
                '- Nonetheless\n' ||
                '- Notwithstanding\n' ||
                '- Thus\n' ||
                '- Undoubtedly\n' ||
                'Instead, use clear and straightforward language to convey your feedback.'
            )
        ),
        host => 'http://ollama:11434'
    ) -> 'message' ->> 'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;


### V3 (Final)


In [ ]:
CREATE OR REPLACE FUNCTION generate_code_review(person_name TEXT, task_title TEXT, query_code TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    WITH relevant_tasks AS (
        SELECT title, chunk
        FROM task_instruction_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    relevant_solutions AS (
        SELECT title, chunk
        FROM code_solution_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    relevant_positives AS (
        SELECT title, chunk
        FROM positives_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    relevant_improvements AS (
        SELECT title, chunk
        FROM improvements_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    ),
    relevant_overall AS (
        SELECT title, chunk
        FROM overall_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_code)
        LIMIT 3
    )
    SELECT string_agg(title || ': ' || chunk, E'\n') 
    INTO context_chunks
    FROM (
        SELECT * FROM relevant_tasks
        UNION ALL
        SELECT * FROM relevant_solutions
        UNION ALL
        SELECT * FROM relevant_positives
        UNION ALL
        SELECT * FROM relevant_improvements
        UNION ALL
        SELECT * FROM relevant_overall
    ) AS combined_context;

    IF context_chunks IS NULL OR context_chunks = '' THEN
        RAISE EXCEPTION 'No relevant context found for the provided query_code.';
    END IF;

    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
            jsonb_build_object(
                'role', 'system',
                'content', 'You are Branden, a friendly and practical code reviewer. Provide clear, actionable feedback with a mix of professionalism and casual tone that is personal and tailored to the student.'
            ),
            jsonb_build_object(
                'role', 'user',
                'content',
                'Code to Review:\n' || query_code ||
                E'\n\nReview Context:\n' || context_chunks ||
                E'Start the Code Review with a friendly introduction.\n' ||
                'Hey, ' || person_name || ', thank you for completing your task on' || task_title ||
                '\n\nRubric Instructions:\n' ||
                '1. Efficiency: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                '2. Completeness: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                '3. Style: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                '4. Documentation: Rate as Unsatisfactory, Needs Work, Acceptable, or Outstanding.\n' ||
                '\n\nCode Review Structure:\n' ||
                '1. Positive aspects of the submission\n' ||
                '2. Aspects that could be improved\n' ||
                '3. Overall comments\n' ||
                'Generate a clear and comprehensive review following this structure:\n' ||
                'Ensure that relevant task objectives are successfully implemented and the student as followed the task instructions\n' ||
                'Include code snippets where relevant\n' ||
                'Include resource links to external resources that are relevant to the code review.\n' ||
                '\n\nPhrases And Words To Avoid:\n'||
                'Phrases:\n' ||
                '- A testament to…\n' ||
                '- In conclusion…\n' ||
                '- In summary…\n' ||
                '- Its important to note/consider…\n' ||
                '- Its worth noting that…\n' ||
                '- On the contrary…\n' ||
                'Words:\n' ||
                '- Aligns\n' ||
                '- Augment\n' ||
                '- Delve\n' ||
                '- Embark\n' ||
                '- Facilitate\n' ||
                '- Maximize\n' ||
                '- Underscores\n' ||
                '- Utilize\n' ||
                '- Efficiency\n' ||
                '- Innovation\n' ||
                '- Institution\n' ||
                '- Integration\n' ||
                '- Implementation\n' ||
                '- Landscape\n' ||
                '- Optimization\n' ||
                '- Realm\n' ||
                '- Tapestry\n' ||
                '- Transformation\n' ||
                '- Adept\n' ||
                '- Commendable\n' ||
                '- Dynamic\n' ||
                '- Efficient\n' ||
                '- Ever-evolving\n' ||
                '- Exciting\n' ||
                '- Exemplary\n' ||
                '- Innovative\n' ||
                '- Invaluable\n' ||
                '- Robust\n' ||
                '- Seamless\n' ||
                '- Synergistic\n' ||
                '- Thought-provoking\n' ||
                '- Transformative\n' ||
                '- Utmost\n' ||
                '- Vibrant\n' ||
                '- Vital\n' ||
                'Additionally, avoid using the following adverbs:\n' ||
                '- Accordingly\n' ||
                '- Additionally\n' ||
                '- Arguably\n' ||
                '- Certainly\n' ||
                '- Consequently\n' ||
                '- Hence\n' ||
                '- However\n' ||
                '- Indeed\n' ||
                '- Moreover\n' ||
                '- Nevertheless\n' ||
                '- Nonetheless\n' ||
                '- Notwithstanding\n' ||
                '- Thus\n' ||
                '- Undoubtedly'             
            )
        ),
        host => 'http://ollama:11434'
    ) -> 'message' ->> 'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;


### Generate RAG responses


In [ ]:
CREATE OR REPLACE FUNCTION generate_rag_response_tasks(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    -- Combine and rank relevant chunks from code reviews and solution files
    WITH ranked_chunks AS (
        SELECT 
            'Code Review Sentiment' AS source, 
            chunk, 
            embedding <=> ai.ollama_embed('nomic-embed-text', query_text) AS distance
        FROM code_review_sentiment_embeddings
        UNION ALL
        SELECT 
            'Code Review Text' AS source, 
            chunk, 
            embedding <=> ai.ollama_embed('nomic-embed-text', query_text) AS distance
        FROM code_review_text_embeddings
        UNION ALL
        SELECT 
            'Solution File' AS source, 
            chunk, 
            embedding <=> ai.ollama_embed('nomic-embed-text', query_text) AS distance
        FROM code_solution_embeddings
        UNION ALL
        SELECT 
            'Solution Filename' AS source, 
            chunk, 
            embedding <=> ai.ollama_embed('nomic-embed-text', query_text) AS distance
        FROM solution_filename_embeddings
        ORDER BY distance ASC
    )
    SELECT STRING_AGG(chunk, E'\n') 
    INTO context_chunks
    FROM ranked_chunks;

    -- Generate a response using the AI model
    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
            jsonb_build_object('role', 'system', 'content', 'You are a helpful assistant. Provide accurate and concise answers.'),
            jsonb_build_object(
                'role', 'user',
                'content', query_text || E'\nUse the following context to respond:\n' || context_chunks
            )
        )
    )->'message'->>'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;


#### Limited Context - Tasks and Code Solutions


In [ ]:
CREATE OR REPLACE FUNCTION generate_rag_response_tasks_solutions(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    -- Retrieve all relevant task titles
    WITH relevant_titles AS (
        SELECT title, chunk
        FROM task_title_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 10
    ),
    -- Retrieve all relevant task instructions
    relevant_tasks AS (
        SELECT title, chunk
        FROM task_instruction_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 10
    ),
    -- Retrieve all relevant code solutions
    relevant_solutions AS (
        SELECT title, chunk
        FROM code_solution_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 10
    )
    -- Combine the results into a single context with explicit labels
    SELECT string_agg(
        CASE 
            WHEN source = 'titles' THEN 'Task Titles:\n' || chunk
            WHEN source = 'instructions' THEN 'Task Instructions:\n' || chunk
            WHEN source = 'solutions' THEN 'Code Solutions:\n' || chunk
        END, E'\n\n'
    ) 
    INTO context_chunks
    FROM (
        SELECT 'titles' AS source, title || ': ' || chunk AS chunk FROM relevant_titles
        UNION ALL
        SELECT 'instructions' AS source, title || ': ' || chunk AS chunk FROM relevant_tasks
        UNION ALL
        SELECT 'solutions' AS source, title || ': ' || chunk AS chunk FROM relevant_solutions
    ) AS combined_context;

    -- Generate a summary using llama3
    SELECT ai.ollama_chat_complete
    ( 'llama3'
    , jsonb_build_array
        ( jsonb_build_object('role', 'system', 'content', 'you are a helpful assistant')
        , jsonb_build_object
        ('role', 'user'
        , 'content', query_text || E'\nUse the following context to respond.\n' || context_chunks
        )
        )
    )->'message'->>'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;


In [ ]:
SELECT generate_rag_response_tasks_solutions('List task titles and the associated task instructions.');

#### All context

In [ ]:
CREATE OR REPLACE FUNCTION generate_rag_response(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    -- Retrieve relevant task instructions
    WITH relevant_tasks AS (
        SELECT title, chunk
        FROM task_instruction_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 3
    ),
    -- Retrieve relevant code solutions
    relevant_solutions AS (
        SELECT title, chunk
        FROM code_solution_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 3
    ),
    -- Retrieve relevant positives from code reviews
    relevant_positives AS (
        SELECT title, chunk
        FROM positives_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 3
    ),
    -- Retrieve relevant improvements from code reviews
    relevant_improvements AS (
        SELECT title, chunk
        FROM improvements_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 3
    ),
    -- Retrieve overall reviews from code reviews
    relevant_overall AS (
        SELECT title, chunk
        FROM overall_code_review_embeddings
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 3
    )
    -- Combine the results into a single context
    SELECT string_agg(title || ': ' || chunk, E'\n') 
    INTO context_chunks
    FROM (
        SELECT * FROM relevant_tasks
        UNION ALL
        SELECT * FROM relevant_solutions
        UNION ALL
        SELECT * FROM relevant_positives
        UNION ALL
        SELECT * FROM relevant_improvements
        UNION ALL
        SELECT * FROM relevant_overall
    ) AS combined_context;

    -- Generate a summary using llama3
    SELECT ai.ollama_chat_complete
    ( 'llama3'
    , jsonb_build_array
        ( jsonb_build_object('role', 'system', 'content', 'you are a helpful assistant')
        , jsonb_build_object
        ('role', 'user'
        , 'content', query_text || E'\nUse the following context to respond.\n' || context_chunks
        )
        )
    )->'message'->>'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;

In [ ]:
SELECT generate_rag_response('What task titles are you aware of?');